In [1]:
#!pip install beautifulsoup4 requests selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00


Collecting Signal IDs

In [18]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

# The base URL of the website we want to crawl
base_url = 'https://www.mql5.com/en/signals/mt5/list/page'

# The list of pages we want to crawl // Useless code 
page_urls = ['1']

# The CSV file we want to write the data to
csv_file = open('signal_list.csv', 'w', newline='')
writer = csv.writer(csv_file)
writer.writerow(['Signal Id'])

# Loop over each page URL and extract the Signal ID data
for page_url in page_urls:
    # Construct the full URL for the page
    url = base_url + page_url

    # Make a GET request to the page
    response = requests.get(url)

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the element has id 
    ids = soup.find_all('div', class_='button button_white-and-green')

    #extrac the id
    for id in ids:
        name = id.get("data-id")
        # Write the data to the CSV file
        writer.writerow([name])

# Close the CSV file
csv_file.close()


Viewing IDs List

In [19]:
import pandas as pd
IDs = pd.read_csv("signal_list.csv")
IDs


,Signal Id
0,1368342
1,1636506
2,843451
3,1486477
4,1818780
5,1598419
6,1245170
7,1722050
8,1725375
9,1334849


In [24]:
import re
base_url_singleID = 'https://www.mql5.com/en/signals/'

#load the csv containing Signal IDs
with open('signal_details.csv', 'w', newline='') as signal_detailsFile:
  writer = csv.writer(signal_detailsFile)
  writer.writerow(['Signal Id','Trades','ProfitTrades','LossTrades','BestTrades','WorstTrades','GrossProfit','GrossLoss','SharpeRatio','RecoveryFactor','ProfitFactor','ExpectedPayoff'])
  for row in IDs['Signal Id']:
    url = base_url_singleID + str(row) + "?source=Unknown#!tab=stats"
    
    #Crawl each signal
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    values = soup.find_all('div', class_='s-data-columns__value')

    # Loop over each product and extract the data
    signalTrades = values[0].text 


    # Get total number of profit trades
    s = values[1].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalProfitTrades = normalized_number

    # Get total number of loss trades

    s = values[2].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalLossTrades = normalized_number

    signalBestTrade = values[3].find('i').text
    signalWorstTrade = values[4].find('i').text

    signalGrossProfit = values[5].find('i').text
    signalGrossLoss = values[6].find('i').text

    signalSharpeRatio = values[9].text

    signalRecoveryFactor = values[15].text


    signalProfitFactor = values[18].text

    signalExpectedPayoff = values[19].find('i').text
    writer.writerow([row,signalTrades,signalProfitTrades,signalLossTrades,signalBestTrade,signalWorstTrade,signalGrossProfit,signalGrossLoss,signalSharpeRatio,signalRecoveryFactor,signalProfitFactor,signalExpectedPayoff])
    




TypeError: 'int' object is not subscriptable

Load the Ids

Test one id

In [51]:
response = requests.get("https://www.mql5.com/en/signals/1909561?source=Unknown#!tab=stats")
soup = BeautifulSoup(response.content, 'html.parser')
values = soup.find_all('div', class_='s-data-columns__value')

    # Loop over each product and extract the data
for value in values:
  print(value.text)

520
507 (97.50%)
13 (2.50%)

183.04 USD
                  

-3 168.30 USD
                  

8 879.70 USD
 (403 489 pips)                  

-9 029.53 USD
 (73 820 pips)                  
125 (1 846.61 USD)
2 284.05 USD (98)
0.03
100.00%
15.64%
4 days ago
14
2 days
-0.02
386 (74.23%)
134 (25.77%)
0.98

-0.29 USD
                  

17.51 USD
                  

-694.58 USD
                  
4 (-7 636.15 USD)
-7 636.15 USD (4)

-27.05%


-100.00%

0%

183.04 USD
                    
125 (1 846.61 USD)
2 284.05 USD (98)

-3 168.30 USD
                    
4 (-7 636.15 USD)
-7 636.15 USD (4)

1 028.03 USD
                    
7 636.15 USD (112.15%)
35.55% (7 642.76 USD)
43.97% (9 449.76 USD)


Manual Data Gathering 

In [52]:
import re


# Get total number of trades
signalTrades = values[0].text 


# Get total number of profit trades
s = values[1].text
number_regex = r"([\d\s]+)\s*\(.*\)"
match = re.search(number_regex, s)
if match:
    number = match.group(1)
    # Remove spaces and normalize the number
    normalized_number = float(number.replace(" ", "").replace(",", ""))
signalProfitTrades = normalized_number

# Get total number of loss trades

s = values[2].text
number_regex = r"([\d\s]+)\s*\(.*\)"
match = re.search(number_regex, s)
if match:
    number = match.group(1)
    # Remove spaces and normalize the number
    normalized_number = float(number.replace(" ", "").replace(",", ""))
signalLossTrades = normalized_number

signalBestTrade = values[3].find('i').text
signalWorstTrade = values[4].find('i').text

signalGrossProfit = values[5].find('i').text
signalGrossLoss = values[6].find('i').text

signalSharpeRatio = values[9].text

signalRecoveryFactor = values[15].text


signalProfitFactor = values[18].text

signalExpectedPayoff = values[19].find('i').text



In [53]:
import re

s = "1 306 (70.63%)"
number_regex = r"([\d\s]+)\s*\(.*\)"
match = re.search(number_regex, s)
if match:
    number = match.group(1)
    # Remove spaces and normalize the number
    normalized_number = float(number.replace(" ", "").replace(",", ""))

print(normalized_number)

1306.0
